In [ ]:
%matplotlib inline

import time

import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MousePosition, BeautifyIcon
import math
from shapely.geometry import Point, Polygon

import json

In [ ]:
with open('../../data/loops.json', 'r') as loopsFile:
    data = loopsFile.read()

# parse file
loops = json.loads(data)
loops

In [ ]:
def loop_to_polygon(loop):
    df = pd.DataFrame.from_dict(loop)
    return Polygon(zip(df['lon'], df['lat']))

def loop_list_to_geo(loop_list):
    polygons = [loop_to_polygon(loop) for loop in loop_list]

    geocoded = gpd.GeoDataFrame(crs='epsg:4326', geometry=polygons)
    return geocoded    

def loop_list_map(loop_list, m=None, style={}):
    geocoded = loop_list_to_geo(loop_list)
    
    new_m = m or folium.Map(location=[-25.2744, 133.7751], zoom_start=4, tiles='CartoDB positron')
    folium.GeoJson(geocoded, style_function=lambda x: style).add_to(new_m)

    return new_m

In [ ]:
loop_list_map(loops)

In [ ]:
def get_lowest_left(loop):
    p = loop[0]
    for point in loop:
        if point['lat'] < p['lat']:
            p = point
        elif point['lat'] == p['lat'] and point['lon'] < p['lon']:
            p = point
    return p

def get_polar_angle(p0, p1):
    return math.atan2(p1['lat'] - p0['lat'], p1['lon'] - p0['lon'])

def sq(n):
    return n * n

def get_sq_distance(p0, p1):
    return sq(p1['lat'] - p0['lat']) + sq(p1['lon'] - p0['lon'])

def get_polar_points(p0, loop):
    polar_points = [{
        **point, 
        'ang': get_polar_angle(p0, point),
        'dist': get_sq_distance(p0, point)
    } for point in loop]

    furthest_polar_points = dict()
    for point in polar_points:
        if point['ang'] in furthest_polar_points:
            if point['dist'] > furthest_polar_points[point['ang']]['dist']:
                furthest_polar_points[point['ang']] = point
        else:
            furthest_polar_points[point['ang']] = point
    
    sorted_points = [v for k, v in sorted(furthest_polar_points.items(), key=lambda s: s[0])]
    return sorted_points

def ccw(p0, p1, p2):
    return (p1['lon'] - p0['lon']) * (p2['lat'] - p0['lat']) - (p2['lon'] - p0['lon']) * (p1['lat'] - p0['lat']); 

def graham_scan(loop):
    stack = []
    p0 = get_lowest_left(loop)
    
    polar_points = get_polar_points(p0, loop)
    for point in polar_points:
        while len(stack) > 1 and ccw(stack[-2], stack[-1], point) <= 0:
            del stack[-1]
        stack.append(point)
        
    return stack

In [ ]:
envelopes = [graham_scan(loop) for loop in loops]
#envelopes = [graham_scan(loops[0])]

In [ ]:
print(f'Original: {len(loops[0])}')
print(f'Envelope: {len(envelopes[0])}')
compare_map = loop_list_map(loops, style={'fill': False})
compare_map = loop_list_map(envelopes, compare_map, {'color': '#ff0000', 'fill': False})
compare_map

In [ ]:
def side_of_line(p0, p1, point):
    return (point['lat'] - p0['lat']) * (p1['lon'] - p0['lon']) - (point['lon'] - p0['lon']) * (p1['lat'] - p0['lat'])

def inside_convex(convex, point):
    overall_side = 0
    for idx, p in enumerate(convex):
        side = side_of_line(p, convex[(idx + 1) % len(convex)], point)
        
        if side == 0:
            return True
        
        if idx == 0:
            overall_side = side > 0
        elif (side > 0) != overall_side:
            return False
    return True

def crossing_direction(p0, p1, point):
    if point['lat'] >= p0['lat'] and point['lat'] < p1['lat']:
        return -1
    if point['lat'] >= p1['lat'] and point['lat'] < p0['lat']:
        return 1
    return 0

def inside_complex(polygon, point):
    winding_order = 0
    for idx, p in enumerate(polygon):
        side = side_of_line(p, polygon[(idx + 1) % len(polygon)], point)
        if side == 0:
            return True
        
        crossing = crossing_direction(p, polygon[(idx + 1) % len(polygon)], point)
        if crossing != 0:
            if crossing == 1 and side > 0:
                winding_order = winding_order + 1
            elif crossing == -1 and side < 0:
                winding_order = winding_order - 1

    return winding_order != 0

def get_bounds(loop):
    minlat = loop[0]['lat']
    minlon = loop[0]['lon']
    maxlat = loop[0]['lat']
    maxlon = loop[0]['lon']
    
    for p in loop:
        if p['lat'] < minlat: minlat = p['lat']
        if p['lat'] > maxlat: maxlat = p['lat']
        if p['lon'] < minlon: minlon = p['lon']
        if p['lon'] > maxlon: maxlon = p['lon']
            
    return [minlat, maxlat, minlon, maxlon]

def point_inside_bounds(bounds, point):
    return (
        point['lat'] >= bounds[0] and
        point['lat'] <= bounds[1] and
        point['lon'] >= bounds[2] and
        point['lon'] <= bounds[3]
    )

def inside_bounds(bounds, point):
    return point_inside_bounds(bounds, point)

In [ ]:
def get_icon(is_inside):
    if is_inside:
        return BeautifyIcon("check", icon_shape="marker", border_color="#0f0", text_color="#0f0")
    else:
        return BeautifyIcon("times", icon_shape="marker", border_color="#f00", text_color="#f00")

test_points = [
    {'lat': -35.2975906, 'lon': 149.1012676},
    {'lat': -39.23225, 'lon': 158.37891},
    envelopes[0][0],
    
    {'lat': envelopes[0][100]['lat'], 'lon': envelopes[0][100]['lon']},
    {'lat': envelopes[0][100]['lat'], 'lon': envelopes[0][100]['lon'] - 0.01},
    {'lat': envelopes[0][100]['lat'], 'lon': envelopes[0][100]['lon'] + 0.01},
    
    {'lat': envelopes[0][150]['lat'], 'lon': envelopes[0][150]['lon']},
    {'lat': envelopes[0][150]['lat'], 'lon': envelopes[0][150]['lon'] - 0.0001},
    {'lat': envelopes[0][150]['lat'], 'lon': envelopes[0][150]['lon'] + 0.0001},
    
    {'lat': -39.77477, 'lon': 145.63477},
]

minlat = -55.3228175
minlon = 72.2461932
maxlat = -9.0880125
maxlon = 168.2261259

n = 10
for i in range(n):
    for j in range(n):
        test_points.append({
            'lat': minlat + ((maxlat - minlat) / n) * i,
            'lon': minlon + ((maxlon - minlon) / n) * j
        })

In [ ]:
point_test_map = loop_list_map([envelopes[0]], style={'fill': False})
MousePosition(position='topright', separator=' | ', prefix="Mouse:",).add_to(point_test_map)

bounds = get_bounds(envelopes[0])
for point in test_points:
    is_inside = inside_bounds(bounds, point)
    
    folium.Marker(location=(point['lat'], point['lon']), icon=get_icon(is_inside)).add_to(point_test_map)

point_test_map

In [ ]:
point_test_map = loop_list_map([envelopes[0]], style={'fill': False})
MousePosition(position='topright', separator=' | ', prefix="Mouse:",).add_to(point_test_map)

for point in test_points:
    is_inside = inside_convex(envelopes[0], point)
    
    folium.Marker(location=(point['lat'], point['lon']), icon=get_icon(is_inside)).add_to(point_test_map)

point_test_map

In [ ]:
point_test_map = loop_list_map([envelopes[0]], style={'fill': False})
MousePosition(position='topright', separator=' | ', prefix="Mouse:",).add_to(point_test_map)

for point in test_points:
    is_inside = inside_complex(envelopes[0], point)
    
    folium.Marker(location=(point['lat'], point['lon']), icon=get_icon(is_inside)).add_to(point_test_map)

point_test_map

In [ ]:
point_test_map = loop_list_map([loops[0]], style={'fill': False})
MousePosition(position='topright', separator=' | ', prefix="Mouse:",).add_to(point_test_map)

for point in test_points:
    is_inside = inside_complex(loops[0], point)
    
    folium.Marker(location=(point['lat'], point['lon']), icon=get_icon(is_inside)).add_to(point_test_map)
    
point_test_map

In [ ]:
# Unfair if other funcs have pre-calculated boundaries
bounds = get_bounds(loops[0])

def test_algo(name, algo_funcs, test_points, boundaries, timings):
    start_time = time.time()
    
    for point in test_points:
        for idx, algo in enumerate(algo_funcs):
            if not algo(boundaries[idx], point):
                break
        
    timings = timings.append({
        'Title': name, 
        'Time': (time.time() - start_time),
        'Boundary Count': len(boundaries[0]) if len(boundaries) == 1 else None,
        'Test Point Count': len(test_points),
    }, ignore_index=True)
    return timings

def test_algos(point_count, timings):
    timing_test_points = []

    for i in range(point_count):
        for j in range(point_count):
            timing_test_points.append({
#                 'lat': bounds[0] + ((bounds[1] - bounds[0]) / point_count) * i,
#                 'lon': bounds[2] + ((bounds[3] - bounds[2]) / point_count) * j
                'lat': -90 + (180 / point_count) * i,
                'lon': -180 + (360 / point_count) * j
            })
            
    timings = test_algo('Bounds', [inside_bounds], timing_test_points, [bounds], timings)
    timings = test_algo('Convex Envelope', [inside_convex], timing_test_points, [envelopes[0]], timings)
    timings = test_algo('Complex Envelope', [inside_complex], timing_test_points, [envelopes[0]], timings)
    timings = test_algo('Complex Loop', [inside_complex], timing_test_points, [loops[0]], timings)
    timings = test_algo('Convex then Complex', [inside_convex, inside_complex], timing_test_points, [envelopes[0], loops[0]], timings)
    timings = test_algo('Bounds then Convex', [inside_bounds, inside_convex], timing_test_points, [bounds, envelopes[0]], timings)
    timings = test_algo('Bounds then Complex', [inside_bounds, inside_complex], timing_test_points, [bounds, loops[0]], timings)
    timings = test_algo('Bounds then Convex then Complex', [inside_bounds, inside_convex, inside_complex], timing_test_points, [bounds, envelopes[0], loops[0]], timings)
    
    return timings
       
timings = pd.DataFrame(
    columns=['Title', 'Time', 'Boundary Count', 'Test Point Count']
)
overall_start_time = time.time()

timings = test_algos(10, timings)
timings = test_algos(20, timings)
timings = test_algos(50, timings)
timings = test_algos(200, timings)

print(f"Execution time: {time.time() - overall_start_time} seconds")
timings

In [ ]:
bounds = get_bounds(loops[0])

timings2 = pd.DataFrame(
    columns=['Title', 'Time', 'Boundary Count', 'Test Point Count']
)
timing_test_points2 = []

point_count = 200
for i in range(point_count):
    for j in range(point_count):
        timing_test_points2.append({
            'lat': bounds[0] + ((bounds[1] - bounds[0]) / point_count) * i,
            'lon': bounds[2] + ((bounds[3] - bounds[2]) / point_count) * j
        })

timings2 = test_algo('Bounds', [inside_bounds], timing_test_points2, [bounds], timings2)

timings2